## Copies data from blob to local host

##### Copyright (C) Microsoft Corporation.  
see license file for details 

In [1]:
# Allow multiple displays per cell

from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all" 

In [2]:
%load_ext autoreload
%autoreload 2

In [3]:
%reload_ext dotenv
dotenv_file_path='./../not_shared/general.env'

In [13]:
import os, platform
import sys
import urllib.request

In [5]:
platform.platform()
os.getcwd()

'Linux-4.15.0-1063-azure-x86_64-with-debian-stretch-sid'

'/datadrive01/prj/AzureChestXRayNoAML/code/01_DataPrep'

In [6]:
# import utlity functions

paths_to_append = [os.path.join(os.getcwd(), os.path.join(*(['..','src'])))]
paths_to_append
def add_path_to_sys_path(path_to_append):
    if not (any(path_to_append in paths for paths in sys.path)):
        sys.path.append(path_to_append)
[add_path_to_sys_path(crt_path) for crt_path in paths_to_append]

import azure_chestxray_utils

['/datadrive01/prj/AzureChestXRayNoAML/code/01_DataPrep/../src']

[None]

/data/anaconda/envs/py35/lib/python3.5/importlib/_bootstrap.py:222: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)
/data/anaconda/envs/py35/lib/python3.5/importlib/_bootstrap.py:222: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)


In [7]:
# create the file path variables 
# create nih_chest_xray_data_dir (in container dir mapped to a host dir for data persistence), 
# where data will be copied from blob

prj_consts = azure_chestxray_utils.chestxray_consts()

nih_chest_xray_data_dir=os.path.join(os.path.join(*([os.sep]+prj_consts.BASE_INPUT_DIR_list)), 
                                     os.path.join(*(prj_consts.ChestXray_IMAGES_DIR_list)))
os.makedirs(nih_chest_xray_data_dir, exist_ok=True)

nih_chest_xray_data_dir
!ls -l {nih_chest_xray_data_dir}

'/data_dir/chestxray/ChestX-ray8/ChestXray-NIHCC'

total 4
drwxr-xr-x 2 root root 4096 Nov 16 05:48 tmp


In [8]:
%dotenv $dotenv_file_path

crt_container = 'https://'+os.getenv('sa_name')+'.blob.core.windows.net/'+os.getenv('sa_container_name')
crt_key = os.getenv('sa_key')

# crt_container
# crt_key

In [9]:
#check azcopy is available. Uncomment second line to print _response_ if needed

response = !azcopy
response[1:10]

['azcopy 7.2.0-netcore Copyright (c) 2018 Microsoft Corp. All Rights Reserved.',
 '------------------------------------------------------------------------------',
 '# azcopy is designed for high-performance uploading, downloading, and copying',
 'data to and from Microsoft Azure Blob, and File storage.',
 '',
 '# Command Line Usage:',
 '    azcopy --source <source> --destination <destination> [options]',
 '',
 '# Options:']

In [10]:
tmp_dir = os.path.join(nih_chest_xray_data_dir, 'tmp')
tmp_dir
os.makedirs(tmp_dir, exist_ok=True)

'/data_dir/chestxray/ChestX-ray8/ChestXray-NIHCC/tmp'

In [11]:
# https://nihcc.app.box.com/v/ChestXray-NIHCC/file/371647823217
# batch_download_zips.py

# Download the 56 zip files in Images_png in batches

# URLs for the zip files
links = [
    'https://nihcc.box.com/shared/static/vfk49d74nhbxq3nqjg0900w5nvkorp5c.gz',
    'https://nihcc.box.com/shared/static/i28rlmbvmfjbl8p2n3ril0pptcmcu9d1.gz',
    'https://nihcc.box.com/shared/static/f1t00wrtdk94satdfb9olcolqx20z2jp.gz',
	'https://nihcc.box.com/shared/static/0aowwzs5lhjrceb3qp67ahp0rd1l1etg.gz',
    'https://nihcc.box.com/shared/static/v5e3goj22zr6h8tzualxfsqlqaygfbsn.gz',
	
	'https://nihcc.box.com/shared/static/asi7ikud9jwnkrnkj99jnpfkjdes7l6l.gz',
	'https://nihcc.box.com/shared/static/jn1b4mw4n6lnh74ovmcjb8y48h8xj07n.gz',
    'https://nihcc.box.com/shared/static/tvpxmn7qyrgl0w8wfh9kqfjskv6nmm1j.gz',
	'https://nihcc.box.com/shared/static/upyy3ml7qdumlgk2rfcvlb9k6gvqq2pj.gz',
	'https://nihcc.box.com/shared/static/l6nilvfa9cg3s28tqv1qc1olm3gnz54p.gz',
	'https://nihcc.box.com/shared/static/hhq8fkdgvcari67vfhs7ppg2w6ni4jze.gz',
	'https://nihcc.box.com/shared/static/ioqwiy20ihqwyr8pf4c24eazhh281pbu.gz'
]

for idx, link in enumerate(links):
    fn = 'images_%02d.tar.gz' % (idx+1)
    fn = os.path.join(tmp_dir, fn)
    print( 'downloading {} to {} ...'.format(link, fn))
    urllib.request.urlretrieve(link, fn)  
print ("Download complete. Please check the checksums")

downloading https://nihcc.box.com/shared/static/vfk49d74nhbxq3nqjg0900w5nvkorp5c.gz to /data_dir/chestxray/ChestX-ray8/ChestXray-NIHCC/tmp/images_01.tar.gz ...


('/data_dir/chestxray/ChestX-ray8/ChestXray-NIHCC/tmp/images_01.tar.gz',
 <http.client.HTTPMessage at 0x7fde57d3e320>)

downloading https://nihcc.box.com/shared/static/i28rlmbvmfjbl8p2n3ril0pptcmcu9d1.gz to /data_dir/chestxray/ChestX-ray8/ChestXray-NIHCC/tmp/images_02.tar.gz ...


('/data_dir/chestxray/ChestX-ray8/ChestXray-NIHCC/tmp/images_02.tar.gz',
 <http.client.HTTPMessage at 0x7fde57d3e710>)

downloading https://nihcc.box.com/shared/static/f1t00wrtdk94satdfb9olcolqx20z2jp.gz to /data_dir/chestxray/ChestX-ray8/ChestXray-NIHCC/tmp/images_03.tar.gz ...


...

downloading https://nihcc.box.com/shared/static/hhq8fkdgvcari67vfhs7ppg2w6ni4jze.gz to /data_dir/chestxray/ChestX-ray8/ChestXray-NIHCC/tmp/images_11.tar.gz ...


('/data_dir/chestxray/ChestX-ray8/ChestXray-NIHCC/tmp/images_11.tar.gz',
 <http.client.HTTPMessage at 0x7fde57d3e320>)

downloading https://nihcc.box.com/shared/static/ioqwiy20ihqwyr8pf4c24eazhh281pbu.gz to /data_dir/chestxray/ChestX-ray8/ChestXray-NIHCC/tmp/images_12.tar.gz ...


('/data_dir/chestxray/ChestX-ray8/ChestXray-NIHCC/tmp/images_12.tar.gz',
 <http.client.HTTPMessage at 0x7fde57d3e898>)

Download complete. Please check the checksums


In [16]:
#NIH ChestXray data at app.box.com
# https://nihcc.app.box.com/v/ChestXray-NIHCC/folder/36938765345
#click on Data_Entry_2017.csv and 'BBox_List_2017.csv' to see their links
tmp_dir1 = os.path.join(nih_chest_xray_data_dir, '..')
link = 'https://nihcc.app.box.com/v/ChestXray-NIHCC/file/219760887468'
fn = 'Data_Entry_2017.csv'
fn = os.path.join(tmp_dir1, fn)
print( 'downloading {} to {} ...'.format(link, fn))
urllib.request.urlretrieve(link, fn)


link = 'https://nihcc.app.box.com/v/ChestXray-NIHCC/file/219760940956'
fn = 'BBox_List_2017.csv'
fn = os.path.join(tmp_dir1, fn)
print( 'downloading {} to {} ...'.format(link, fn))
urllib.request.urlretrieve(link, fn)


downloading https://nihcc.app.box.com/v/ChestXray-NIHCC/file/219760887468 to /data_dir/chestxray/ChestX-ray8/ChestXray-NIHCC/../Data_Entry_2017.csv ...


('/data_dir/chestxray/ChestX-ray8/ChestXray-NIHCC/../Data_Entry_2017.csv',
 <http.client.HTTPMessage at 0x7fbf28c18a20>)

downloading https://nihcc.app.box.com/v/ChestXray-NIHCC/file/219760940956 to /data_dir/chestxray/ChestX-ray8/ChestXray-NIHCC/../BBox_List_2017.csv ...


('/data_dir/chestxray/ChestX-ray8/ChestXray-NIHCC/../BBox_List_2017.csv',
 <http.client.HTTPMessage at 0x7fbf28c18c18>)

In [11]:
!ls -l  $tmp_dir


total 44023372
-rw-r--r-- 1 root root 2008470987 Nov 16 05:15 images_01.tar.gz
-rw-r--r-- 1 root root 3952623504 Nov 16 05:19 images_02.tar.gz
-rw-r--r-- 1 root root 3929234850 Nov 16 05:21 images_03.tar.gz
-rw-r--r-- 1 root root 3838903983 Nov 16 05:25 images_04.tar.gz
-rw-r--r-- 1 root root 3935496531 Nov 16 05:29 images_05.tar.gz
-rw-r--r-- 1 root root 3986301172 Nov 16 05:32 images_06.tar.gz
-rw-r--r-- 1 root root 4016328426 Nov 16 05:35 images_07.tar.gz
-rw-r--r-- 1 root root 4018347353 Nov 16 05:39 images_08.tar.gz
-rw-r--r-- 1 root root 4111327929 Nov 16 05:42 images_09.tar.gz
-rw-r--r-- 1 root root 4181556296 Nov 16 05:45 images_10.tar.gz
-rw-r--r-- 1 root root 4187084020 Nov 16 05:48 images_11.tar.gz
-rw-r--r-- 1 root root 2914187733 Nov 16 05:51 images_12.tar.gz


In [14]:
# copy data to local host dir
# add {  --source-key {crt_key}  } if needed

tmp_dir1 = os.path.join(nih_chest_xray_data_dir, '..')

!azcopy --quiet \
    --source {crt_container}  \
    --destination {tmp_dir1} \
    --source-key {crt_key} \
    --verbose \
    --include "BBox_List_2017.csv"

!azcopy --quiet \
    --source {crt_container}  \
    --destination {tmp_dir1} \
    --source-key {crt_key} \
    --verbose \
    --include "Data_Entry_2017.csv"

!azcopy --quiet \
    --source {crt_container}  \
    --destination {tmp_dir1} \
    --source-key {crt_key} \
    --verbose \
    --include "blacklist.csv"

Error: unknown command "https://ghiordanazrchestxraysa.blob.core.windows.net/nihchestxraydata" for "azcopy"
Run 'azcopy --help' for usage.

unknown command "https://ghiordanazrchestxraysa.blob.core.windows.net/nihchestxraydata" for "azcopy"
Error: unknown command "https://ghiordanazrchestxraysa.blob.core.windows.net/nihchestxraydata" for "azcopy"
Run 'azcopy --help' for usage.

unknown command "https://ghiordanazrchestxraysa.blob.core.windows.net/nihchestxraydata" for "azcopy"
Error: unknown command "https://ghiordanazrchestxraysa.blob.core.windows.net/nihchestxraydata" for "azcopy"
Run 'azcopy --help' for usage.

unknown command "https://ghiordanazrchestxraysa.blob.core.windows.net/nihchestxraydata" for "azcopy"


In [13]:
tmp_dir = os.path.join(nih_chest_xray_data_dir, 'tmp')
tmp_dir
os.makedirs(tmp_dir, exist_ok=True)
compressed_data_location = crt_container +'/compresseddata'

'/local_dir/prj/AzureChestXRayNoAML/code/01_DataPrep/../../../../data/chestxray/ChestX-ray8/ChestXray-NIHCC/tmp'

In [14]:
answer = !azcopy --quiet \
    --source {compressed_data_location}  \
    --destination {tmp_dir} \
    --source-key {crt_key} \
    --recursive \
    --verbose \
    --include "images_"

In [15]:
answer[-7:]

['[2019/05/24 05:17:27] Transfer summary:',
 '-----------------',
 'Total files transferred: 12',
 'Transfer successfully:   12',
 'Transfer skipped:        0',
 'Transfer failed:         0',
 'Elapsed time:            00.00:00:50']

The azcopy download process above will run for several minutes. 

Make sure you have these file sizes when it finishes:  
total __44023372__  
-rw-r--r-- 1 1000 users __2008470987__ Sep 26 17:19 images_001.tar.gz  
-rw-r--r-- 1 1000 users __3952623504__ Sep 26 19:05 images_002.tar.gz  
-rw-r--r-- 1 1000 users __3929234850__ Sep 26 17:22 images_003.tar.gz  
-rw-r--r-- 1 1000 users __3838903983__ Sep 26 17:25 images_004.tar.gz  
-rw-r--r-- 1 1000 users __3935496531__ Sep 26 17:22 images_005.tar.gz  
-rw-r--r-- 1 1000 users __3986301172__ Sep 26 17:25 images_006.tar.gz  
-rw-r--r-- 1 1000 users __4016328426__ Sep 26 19:05 images_007.tar.gz  
-rw-r--r-- 1 1000 users __4018347353__ Sep 26 19:05 images_008.tar.gz  
-rw-r--r-- 1 1000 users __4111327929__ Sep 26 19:05 images_009.tar.gz  
-rw-r--r-- 1 1000 users __4181556296__ Sep 26 19:05 images_010.tar.gz  
-rw-r--r-- 1 1000 users __4187084020__ Sep 26 19:05 images_011.tar.gz  
-rw-r--r-- 1 1000 users __2914187733__ Sep 26 17:21 images_012.tar.gz  
  
If the unzipping process below does not end up with __112120__ .png images, you may need to repeat the download process for the above 12 .tar.gz files.

In [16]:
nih_chest_xray_data_dir
tmp_dir
!ls -l $tmp_dir
# !find $nih_chest_xray_data_dir -type f | wc -l

'/local_dir/prj/AzureChestXRayNoAML/code/01_DataPrep/../../../../data/chestxray/ChestX-ray8/ChestXray-NIHCC'

'/local_dir/prj/AzureChestXRayNoAML/code/01_DataPrep/../../../../data/chestxray/ChestX-ray8/ChestXray-NIHCC/tmp'

total 44023372
-rw-r--r-- 1 root root 2008470987 May 24 05:17 images_01.tar.gz
-rw-r--r-- 1 root root 3952623504 May 24 05:17 images_02.tar.gz
-rw-r--r-- 1 root root 3929234850 May 24 05:17 images_03.tar.gz
-rw-r--r-- 1 root root 3838903983 May 24 05:17 images_04.tar.gz
-rw-r--r-- 1 root root 3935496531 May 24 05:17 images_05.tar.gz
-rw-r--r-- 1 root root 3986301172 May 24 05:17 images_06.tar.gz
-rw-r--r-- 1 root root 4016328426 May 24 05:17 images_07.tar.gz
-rw-r--r-- 1 root root 4018347353 May 24 05:17 images_08.tar.gz
-rw-r--r-- 1 root root 4111327929 May 24 05:17 images_09.tar.gz
-rw-r--r-- 1 root root 4181556296 May 24 05:17 images_10.tar.gz
-rw-r--r-- 1 root root 4187084020 May 24 05:17 images_11.tar.gz
-rw-r--r-- 1 root root 2914187733 May 24 05:17 images_12.tar.gz


The unzipping process below will run for several minutes.

While it runs, you can monitor the progress by ssh-ing 
into your compute context machine and running:   
find /datadrive01/data/chestxray/ChestX-ray8/ChestXray-NIHCC/images/  -type f | wc -l  

The final number of images is:  
__112120__  



In [17]:
nih_chest_xray_data_dir
old_crt_dir = os.getcwd()
!ls -l ./../01_DataPrep/../../../../
os.chdir(nih_chest_xray_data_dir)
!pwd
!ls -l $tmp_dir/*.tar.gz 
allImages = ! cat $tmp_dir/*.tar.gz | tar -zxf - -i
# !for file in $tmp_dir/*.tar.gz; do tar -zxf $file; done


'/local_dir/prj/AzureChestXRayNoAML/code/01_DataPrep/../../../../data/chestxray/ChestX-ray8/ChestXray-NIHCC'

total 8
drwxrwxrwx  3 root root 4096 May 24 02:33 data
drwxrwxrwx 10 root root 4096 May 24 05:01 prj
/local_dir/data/chestxray/ChestX-ray8/ChestXray-NIHCC
-rw-r--r-- 1 root root 2008470987 May 24 05:17 /local_dir/prj/AzureChestXRayNoAML/code/01_DataPrep/../../../../data/chestxray/ChestX-ray8/ChestXray-NIHCC/tmp/images_01.tar.gz
-rw-r--r-- 1 root root 3952623504 May 24 05:17 /local_dir/prj/AzureChestXRayNoAML/code/01_DataPrep/../../../../data/chestxray/ChestX-ray8/ChestXray-NIHCC/tmp/images_02.tar.gz
-rw-r--r-- 1 root root 3929234850 May 24 05:17 /local_dir/prj/AzureChestXRayNoAML/code/01_DataPrep/../../../../data/chestxray/ChestX-ray8/ChestXray-NIHCC/tmp/images_03.tar.gz
-rw-r--r-- 1 root root 3838903983 May 24 05:17 /local_dir/prj/AzureChestXRayNoAML/code/01_DataPrep/../../../../data/chestxray/ChestX-ray8/ChestXray-NIHCC/tmp/images_04.tar.gz
-rw-r--r-- 1 root root 3935496531 May 24 05:17 /local_dir/prj/AzureChestXRayNoAML/code/01_DataPrep/../../../../data/chestxray/ChestX-ray8/ChestXr

In [18]:
# os.chdir(nih_chest_xray_data_dir)
# !pwd
# !tar -zxvf $tmp_dir/images_001.tar.gz

In [19]:
os.chdir(old_crt_dir) 
!jupyter nbconvert --to html 001_get_data.ipynb

[NbConvertApp] Converting notebook 001_get_data.ipynb to html
[NbConvertApp] Writing 314567 bytes to 001_get_data.html
